# Initial datasets preparation

## Imports

In [117]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# Definir o número máximo de colunas do pandas para 40
pd.set_option('display.max_columns', 40)

## Merge the datasets

### Train dataset

In [118]:
allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

e1 = pd.read_csv("datasets/old/energia_202109-202112.csv", na_values=allowed_nans, keep_default_na=False)
e2 = pd.read_csv("datasets/old/energia_202201-202212.csv", na_values=allowed_nans, keep_default_na=False)

m1 = pd.read_csv("datasets/old/meteo_202109-202112.csv")
m2 = pd.read_csv("datasets/old/meteo_202201-202212.csv")

e_treino = pd.concat([e1, e2])
m_treino = pd.concat([m1, m2])

m_treino['dt_iso'] = m_treino['dt_iso'].str.replace(' UTC', '')
m_treino['dt_iso'] = pd.to_datetime(m_treino['dt_iso'], format="%Y-%m-%d %H:%M:%S %z")
m_treino['Data'] = m_treino['dt_iso'].dt.strftime('%Y-%m-%d')
m_treino['Hora'] = m_treino['dt_iso'].dt.strftime('%H')
m_treino['Hora'] = m_treino['Hora'].astype(int)

treino = pd.merge(m_treino, e_treino, on=['Data','Hora'], how='inner')

In [119]:
t_sea_grn_pressure = pd.read_csv("datasets/old/seaGrndPrss2909.csv")

t_sea_grn_pressure.rename(columns={"pressure_msl (hPa)": "sea_level", "surface_pressure (hPa)": "grnd_level"}, inplace=True)
treino['sea_level'] = t_sea_grn_pressure['sea_level']
treino['grnd_level'] = t_sea_grn_pressure['grnd_level']

In [120]:
columns_order = ['dt','dt_iso','Data','Hora', 'city_name', 'temp', 'feels_like', 'temp_min', 'temp_max', 
                 'pressure', 'sea_level', 'grnd_level', 'humidity', 'wind_speed', 'rain_1h', 'clouds_all',
                 'weather_description', 'Normal (kWh)', 'Horario Economico (kWh)', 'Autoconsumo (kWh)', 'Injeçao na rede (kWh)']

treino.reindex(columns=columns_order)
treino

,dt,dt_iso,city_name,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,rain_1h,clouds_all,weather_description,Data,Hora,Normal (kWh),Horario Economico (kWh),Autoconsumo (kWh),Injeçao na rede (kWh)
0,1632873600,2021-09-29 00:00:00+00:00,local,13.97,13.54,11.45,14.04,1027,1027.9,1004.4,81,1.96,NaN,87,overcast clouds,2021-09-29,0,0.000,0.000,0.0,None
1,1632877200,2021-09-29 01:00:00+00:00,local,13.48,13.02,13.43,13.90,1028,1027.9,1004.4,82,1.83,NaN,91,overcast clouds,2021-09-29,1,0.000,0.000,0.0,None
2,1632880800,2021-09-29 02:00:00+00:00,local,12.93,12.47,12.72,13.43,1027,1027.8,1004.3,84,1.96,NaN,93,overcast clouds,2021-09-29,2,0.000,0.000,0.0,None
3,1632884400,2021-09-29 03:00:00+00:00,local,12.61,12.15,10.34,12.93,1027,1027.6,1004.1,85,1.85,NaN,95,overcast clouds,2021-09-29,3,0.000,0.000,0.0,None
4,1632888000,2021-09-29 04:00:00+00:00,local,12.61,12.17,9.79,12.93,1027,1027.8,1004.3,86,1.83,NaN,93,overcast clouds,2021-09-29,4,0.000,0.000,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11011,1672513200,2022-12-31 19:00:00+00:00,local,15.70,15.57,15.57,15.72,1018,1018.4,995.3,86,5.56,3.58,100,moderate rain,2022-12-31,19,1.693,0.000,0.0,None
11012,1672516800,2022-12-31 20:00:00+00:00,local,15.55,15.43,12.72,15.70,1018,1018.8,995.7,87,4.47,4.20,100,heavy intensity rain,2022-12-31,20,1.327,0.000,0.0,None
11013,1672520400,2022-12-31 21:00:00+00:00,local,13.45,13.28,12.23,13.99,1019,1019.0,995.9,93,3.29,4.23,100,heavy intensity rain,2022-12-31,21,0.757,0.000,0.0,None
11014,1672524000,2022-12-31 22:00:00+00:00,local,12.93,12.73,12.23,13.43,1019,1019.0,995.9,94,1.49,3.90,100,moderate rain,2022-12-31,22,0.000,0.675,0.0,None


### Test dataset

In [121]:
e_teste = pd.read_csv("datasets/old/energia_202301-202304.csv")
m1_teste = pd.read_csv("datasets/old/meteo_202301-202304.csv")

m_missingData = pd.read_csv("datasets/old/missingMeteoData.csv")

m_teste = pd.concat([m1_teste, m_missingData])
m1_teste
m_teste['dt_iso'] = m_teste['dt_iso'].str.replace(' UTC', '')
m_teste['dt_iso'] = pd.to_datetime(m_teste['dt_iso'], format="%Y-%m-%d %H:%M:%S %z")

m_teste['Data'] = m_teste['dt_iso'].dt.strftime('%Y-%m-%d')
m_teste['Hora'] = m_teste['dt_iso'].dt.strftime('%H')
m_teste['Hora'] = m_teste['Hora'].astype(int)

teste = pd.merge(m_teste, e_teste, on=['Data','Hora'], how='inner')

In [122]:
m_sea_grnd_prss = pd.read_csv("datasets/old/seaGrndPressureBraga2023.csv")

m_sea_grnd_prss.rename(columns={"pressure_msl (hPa)": "sea_level", "surface_pressure (hPa)": "grnd_level"}, inplace=True)
teste['sea_level'] = m_sea_grnd_prss['sea_level']
teste['grnd_level'] = m_sea_grnd_prss['grnd_level']

In [123]:
columns_order = ['dt','dt_iso','Data','Hora', 'city_name', 'temp', 'feels_like', 'temp_min', 'temp_max', 
                 'pressure', 'sea_level', 'grnd_level', 'humidity', 'wind_speed', 'rain_1h', 'clouds_all',
                 'weather_description', 'Normal (kWh)', 'Horario Economico (kWh)', 'Autoconsumo (kWh)']

teste.reindex(columns=columns_order)

,dt,dt_iso,Data,Hora,city_name,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,rain_1h,clouds_all,weather_description,Normal (kWh),Horario Economico (kWh),Autoconsumo (kWh)
0,1.672531e+09,2023-01-01 00:00:00+00:00,2023-01-01,0,local,12.93,12.76,12.72,13.43,1019.0,1019.0,995.8,95,2.02,3.72,100.0,moderate rain,0.000,0.467,0.000
1,1.672535e+09,2023-01-01 01:00:00+00:00,2023-01-01,1,local,13.49,13.38,13.43,13.90,1018.0,1018.1,994.9,95,2.18,3.26,100.0,moderate rain,0.000,0.577,0.000
2,1.672538e+09,2023-01-01 02:00:00+00:00,2023-01-01,2,local,13.55,13.44,13.48,14.82,1017.0,1016.8,993.7,95,2.88,2.44,100.0,moderate rain,0.000,0.346,0.000
3,1.672542e+09,2023-01-01 03:00:00+00:00,2023-01-01,3,local,13.61,13.51,12.01,14.82,1016.0,1016.2,993.1,95,3.63,1.74,100.0,moderate rain,0.000,0.270,0.000
4,1.672546e+09,2023-01-01 04:00:00+00:00,2023-01-01,4,local,13.59,13.49,12.01,14.82,1015.0,1015.3,992.3,95,4.58,1.13,100.0,moderate rain,0.000,0.252,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2251,NaN,2023-04-04 19:00:00+00:00,2023-04-04,19,NaN,20.10,18.10,3.60,20.40,NaN,1015.1,992.6,35,1.20,NaN,0.0,NaN,0.953,0.000,0.012
2252,NaN,2023-04-04 20:00:00+00:00,2023-04-04,20,NaN,13.60,11.50,3.60,20.40,NaN,1016.1,993.1,63,2.47,NaN,0.0,NaN,0.915,0.000,0.000
2253,NaN,2023-04-04 21:00:00+00:00,2023-04-04,21,NaN,12.60,10.60,3.60,20.40,NaN,1017.0,993.9,67,2.22,NaN,2.0,NaN,0.479,0.000,0.000
2254,NaN,2023-04-04 22:00:00+00:00,2023-04-04,22,NaN,11.70,9.50,3.60,20.40,NaN,1017.5,994.3,68,2.41,NaN,1.0,NaN,0.000,0.497,0.000


### Write the datasets to csv

In [124]:
treino.to_csv('datasets/treino.csv', index=False)
teste.to_csv('datasets/teste.csv', index=False)